<a href="https://colab.research.google.com/github/Deeksha-coder-debug/Stock-Prediction-project/blob/main/GRU_Based_Sentiment_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

What is a GRU?
GRU (Gated Recurrent Unit) is a variant of RNN that solves the vanishing gradient problem seen in traditional RNNs. It was introduced as a simpler alternative to LSTMs (Long Short-Term Memory networks). GRUs have two gates: an update gate and a reset gate. These gates help the model decide which information to keep and which to discard over time, allowing it to capture long-term dependencies efficiently without needing a separate memory cell (as used in LSTMs).

Key Features of GRU:
Update Gate: Controls how much of the previous information needs to be passed to the future.
Reset Gate: Determines how much of the previous hidden state should be forgotten or reset.
Simplified Structure: Unlike LSTMs, GRUs have fewer gates and fewer parameters, making them faster to train and less prone to overfitting on small datasets.
GRU vs. LSTM:
GRU: Simpler, fewer parameters, faster to train.
LSTM: More flexible, slightly better at learning long-term dependencies.
GRUs are often chosen when computational efficiency is a priority, especially for tasks like natural language processing.

Step 1: Import Required Libraries and Load IMDb Data

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
# Preloaded IMDb movie review dataset with pre-tokenized reviews.
from tensorflow.keras.preprocessing.sequence import pad_sequences
# to ensure all reviews have the same length for feeding into the GRU.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Embedding, Dense
# Recurrent neural network layer, similar to LSTM but slightly faster.
# Converts integer word indices into dense word vectors (embeddings).

# Parameters
max_features = 10000  # Top 10,000 words
maxlen = 500  # Maximum length of reviews (in terms of words)
# GRU needs fixed-length sequences → ensures uniform input shape.
embedding_size = 128

# Load IMDb dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Step 2: Pad Sequences
The reviews in the dataset vary in length. For neural networks to process the data efficiently, all input sequences must be the same length. We achieve this by padding shorter sequences with zeros using the pad_sequences() function.

In [2]:
# pad sequences to the same length
x_train=pad_sequences(x_train,maxlen=maxlen)
x_test=pad_sequences(x_test,maxlen=maxlen)

By padding all sequences to the same length, the GRU will process each sequence as if it contains exactly 500 time steps, even though many reviews are shorter.

# Step 3: Build a GRU Model
Now, let’s build the GRU-based model. We’ll start with an embedding layer to convert the integer word indices into dense vector representations. The GRU layer follows, which processes the input sequence. Finally, we add a dense layer with a sigmoid activation function to output a probability for binary classification (positive or negative sentiment).

In [3]:
# build GRU model
model_gru=Sequential()
model_gru.add(Embedding(max_features,embedding_size,input_length=maxlen))
model_gru.add(GRU(64))  # Gru layer with 64 units
model_gru.add(Dense(1,activation='sigmoid'))  # output layer for binary classification

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


# Model Architecture Explained:

* **Embedding Layer:** This layer converts word indices into dense vectors of fixed size (embedding_size=128). It allows the model to learn word representations during training.

* ***GRU Layer:*** A single GRU layer with 64 units is used. The number of units controls the capacity of the GRU to learn from sequences. You can increase or decrease this value based on the complexity of the task and dataset.

* **Dense Layer:** The dense layer with a sigmoid activation function outputs a single probability between 0 and 1, representing the sentiment (0 for negative, 1 for positive).

# Step 4: Compile and Train the Model

Next, we compile the model using the Adam optimizer and binary crossentropy loss function. The binary_crossentropy loss is used for binary classification tasks. We will also track the accuracy during training.

In [4]:
# compile model
model_gru.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# train model
model_gru.fit(x_train,y_train,epochs=3,batch_size=64,validation_data=(x_test,y_test))


Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 17s 33ms/step - accuracy: 0.6583 - loss: 0.5918 - val_accuracy: 0.8447 - val_loss: 0.3651
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 17s 31ms/step - accuracy: 0.8914 - loss: 0.2757 - val_accuracy: 0.8692 - val_loss: 0.3254
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 12s 32ms/step - accuracy: 0.9280 - loss: 0.1931 - val_accuracy: 0.8743 - val_loss: 0.3352


**Optimizer:** We use Adam as it is a popular choice for its adaptive learning rate and efficiency.

**Loss Function:** Binary crossentropy is ideal for binary classification problems.

**Metrics:** We track accuracy as the main evaluation metric.

Training:

**Epochs:** We train for 3 epochs. You can experiment with more epochs depending on the dataset and the computational resources available.

**Batch Size**: A batch size of 64 is used. Larger batch sizes typically train faster but consume more memory.

# Step 5: Evaluate the Model

After training, we evaluate the model’s performance on the test dataset to check its accuracy.

In [5]:
# evaluate the model
loss,accuracy=model_gru.evaluate(x_test,y_test)
print(f'GRU Model test accuracy is {accuracy} and loss is {loss}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.8731 - loss: 0.3416
GRU Model test accuracy is 0.8743199706077576 and loss is 0.33519506454467773


# Step 6: Real-Time Text Sentiment Classification
Now that the model is trained, we can use it to predict the sentiment of new movie reviews. We’ll preprocess the text reviews by tokenizing them into sequences of word indices, padding them to the same length as the training data, and passing them through the model for prediction.

In [6]:
#  Sample texts for real-time testing
texts = [
  "I absolutely loved this movie, the storyline was engaging!",
  "The film was too slow and boring for my taste.",
  "A perfect blend of comedy and drama, truly enjoyable!"
]

# Step 7: Preprocess New Texts
We need to convert the new texts into sequences of word indices using the same vocabulary as the IMDb dataset. We achieve this using the Tokenizer from Keras.

The Tokenizer converts the input texts into sequences of word indices that match the format of the training data, and pad_sequences ensures that all sequences are of equal length.

normally, when working with raw text data, the exact workflow is:

**Raw text → Tokenizer → Fit → Word Index → Convert to Sequences → Pad → Model**

But for the IMDb dataset, you must be very careful because Keras has already done part of this pipeline internally

In [7]:
# Load IMDb word index
word_index = tf.keras.datasets.imdb.get_word_index()

# Tokenizer for IMDb dataset
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts([' '.join([str(word) for word in sequence]) for sequence in x_train])

# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(texts)

# Pad the sequences
padded_sequences = pad_sequences(sequences, maxlen=maxlen)

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Step 8: Make Predictions
Finally, we pass the padded sequences to the trained GRU model to predict the sentiment of the texts.

In [10]:
# predict sentiment
predictions=model_gru.predict(padded_sequences)

# output the results
for i,text in enumerate(texts):
  sentiment='Positive' if predictions[i]>0.5 else 'Negative'
  print(f'Text : {text}\nPredicted Sentiment : {sentiment}\n')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Text : I absolutely loved this movie, the storyline was engaging!
Predicted Sentiment : Negative

Text : The film was too slow and boring for my taste.
Predicted Sentiment : Negative

Text : A perfect blend of comedy and drama, truly enjoyable!
Predicted Sentiment : Negative

